In [1]:
from google.colab import drive
drive.mount('/content/drive')
!unzip -q "/content/drive/MyDrive/Data_Challenge/data.zip" -d /content/

Mounted at /content/drive


In [2]:
!git clone -b data_augmentation --single-branch https://github.com/EtiNL/Dropout_disco.git /content/Dropout_disco

%cd content

import sys, os

repo_root = "/content/Dropout_disco"
if not os.path.isdir(repo_root):
    raise FileNotFoundError(f"Repo not found at {repo_root}")

if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

Cloning into '/content/Dropout_disco'...
remote: Enumerating objects: 96, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 96 (delta 51), reused 69 (delta 24), pack-reused 0 (from 0)
Receiving objects: 100% (96/96), 122.55 KiB | 11.14 MiB/s, done.
Resolving deltas: 100% (51/51), done.
[Errno 2] No such file or directory: 'content'
/content


In [3]:
from load_data import load_train_data
from text_encoding import embed_texts
from training import train_clip_with_split
from eval_submission import build_submission_df

# Load data

In [4]:
motion_ids, motion_paths, map_df, text_df = load_train_data("./data", augmented_text=True)

# Encode text

In [5]:
# text_encoder = "Qwen/Qwen3-Embedding-0.6B"
text_encoder = "Qwen/Qwen3-Embedding-4B"

In [ ]:
text_df, text_emb, _ = embed_texts(text_df, model_name=text_encoder)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

Batches:   0%|          | 0/1129 [00:00<?, ?it/s]

In [ ]:
import torch

# Save
torch.save(text_emb, "text_emb_qwen4B.pt")
torch.save(text_df, "text_df_qwen4B.pt")  # or text_df.to_csv("text_df.csv", index=False)

# Download directly from Colab
from google.colab import files
files.download("text_emb_qwen4B.pt")
files.download("text_df_qwen4B.pt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
import torch

text_emb = torch.load("drive/MyDrive/Data_Challenge/text_emb_qwen4B.pt")
text_df = torch.load(
    "drive/MyDrive/Data_Challenge/text_df_qwen4B.pt",
    weights_only=False,
)

# Motion encoder

In [7]:
from motion_encoder.lstm import LSTMMotionEncoder
from motion_encoder.simple_mlp import SimpleMotionEncoder
from motion_encoder.CNN1D import MotionCNN1D
from motion_encoder.gru import GRUMotionEncoder
from motion_encoder.better_motion_encoder import BetterMotionEncoder
from motion_encoder.MDM import MDMMotionEncoder

In [8]:
import numpy as np
x0 = np.load(motion_paths[0])
d_in = x0.shape[1]

# Training

## BetterMotionEncoder

In [ ]:
augs = {
    "translate_xz": {"p": 0.2, "sigma_dx": 0.5, "sigma_dz": 0.5},
    "rotate_y": {"p": 0.5, "sigma_theta": np.deg2rad(60.0)},
}

# motion_model = SimpleMotionEncoder(d_in, hidden=256, out_dim= 512)
motion_model = BetterMotionEncoder(d_in=d_in, d_model=256, out_dim=512, n_res_blocks= 2, dropout=0.1,)

trained_clip, info = train_clip_with_split(
    motion_model=motion_model,
    motion_ids=motion_ids,
    motion_paths=motion_paths,
    map_df=map_df,
    text_df=text_df,
    text_emb=text_emb,
    text_model_name=text_encoder,
    save_path="BetterMotionEncoder_base_aug.pth",
    epochs=150,
    batch_size=128,
    time_padding=True,
    augs=augs,
    patience=50,
    n_val_batches=100,
    weight_decay=1e-3,
    warmup_epochs=5,
    grad_clip=1.0,
    scheduler_cfg={"name": "cosine_wr", "T_0": 30, "T_mult": 2, "eta_min": 1e-6},
)

/content/Dropout_disco/training.py:446: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device == "cuda"))
/content/Dropout_disco/training.py:461: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device == "cuda")):


epoch 001/150 | lr=2.00e-04 | train_loss=4.8554 | val_score=0.0822 | MRR=0.1330 | R@1=0.030 R@2=0.080 R@3=0.130 R@5=0.150 R@10=0.320
Modèle sauvegardé (Nouveau meilleur score : 0.0822)
Found New Best Model! (MRR=0.1330)
epoch 002/150 | lr=4.00e-04 | train_loss=4.6846 | val_score=0.2972 | MRR=0.3376 | R@1=0.130 R@2=0.320 R@3=0.450 R@5=0.600 R@10=0.740
Modèle sauvegardé (Nouveau meilleur score : 0.2972)
Found New Best Model! (MRR=0.3376)
epoch 003/150 | lr=6.00e-04 | train_loss=4.3370 | val_score=0.3638 | MRR=0.4019 | R@1=0.190 R@2=0.400 R@3=0.510 R@5=0.640 R@10=0.880
Modèle sauvegardé (Nouveau meilleur score : 0.3638)
Found New Best Model! (MRR=0.4019)
epoch 004/150 | lr=8.00e-04 | train_loss=4.1569 | val_score=0.3941 | MRR=0.4364 | R@1=0.240 R@2=0.410 R@3=0.500 R@5=0.680 R@10=0.930
Modèle sauvegardé (Nouveau meilleur score : 0.3941)
Found New Best Model! (MRR=0.4364)
epoch 005/150 | lr=1.00e-03 | train_loss=4.0223 | val_score=0.4802 | MRR=0.5136 | R@1=0.340 R@2=0.520 R@3=0.580 R@5=0.70

KeyboardInterrupt: 

## CNN1D

In [ ]:
motion_CNN1D = MotionCNN1D(num_features=384, latent_dim=512)
augs = {
    "translate_xz": {"p": 0.2, "sigma_dx": 0.5, "sigma_dz": 0.5},
    "rotate_y": {"p": 1.0, "sigma_theta": np.deg2rad(60.0)},
}

motion_CNN1D, info = train_clip_with_split(
    motion_model=motion_CNN1D,
    motion_ids=motion_ids,
    motion_paths=motion_paths,
    map_df=map_df,
    text_df=text_df,
    text_emb=text_emb,
    text_model_name=text_encoder,
    save_path="motionCNN1D_512_aug.pth",
    epochs=100,
    batch_size=128,
    time_padding=True,
    augs=augs,
    patience=50,
    n_val_batches=100,
    weight_decay=1e-4,
    warmup_epochs=5,
    grad_clip=1.0,
    scheduler_cfg={"name": "cosine", "T_max": 150, "eta_min": 1e-6},
)

epoch 001/100 | lr=2.00e-04 | train_loss=4.8699 | val_score=0.0570 | MRR=0.1110 | R@1=0.030 R@2=0.050 R@3=0.050 R@5=0.120 R@10=0.260
Modèle sauvegardé (Nouveau meilleur score : 0.0570)
Found New Best Model! (MRR=0.1110)
epoch 002/100 | lr=4.00e-04 | train_loss=4.4530 | val_score=0.3223 | MRR=0.3676 | R@1=0.160 R@2=0.330 R@3=0.470 R@5=0.590 R@10=0.880
Modèle sauvegardé (Nouveau meilleur score : 0.3223)
Found New Best Model! (MRR=0.3676)
epoch 003/100 | lr=6.00e-04 | train_loss=4.2017 | val_score=0.3594 | MRR=0.3996 | R@1=0.190 R@2=0.360 R@3=0.530 R@5=0.670 R@10=0.860
Modèle sauvegardé (Nouveau meilleur score : 0.3594)
Found New Best Model! (MRR=0.3996)
epoch 004/100 | lr=8.00e-04 | train_loss=4.1238 | val_score=0.3448 | MRR=0.3898 | R@1=0.190 R@2=0.340 R@3=0.470 R@5=0.670 R@10=0.850
EarlyStopping counter: 1 out of 50
epoch 005/100 | lr=1.00e-03 | train_loss=4.0736 | val_score=0.3438 | MRR=0.3862 | R@1=0.170 R@2=0.350 R@3=0.490 R@5=0.690 R@10=0.870
EarlyStopping counter: 2 out of 50
epoc

## GRU

In [ ]:
motion_GRU = GRUMotionEncoder(d_in, hidden_dim = 512, out_dim = 1024)

augs = {
    "translate_xz": {"p": 0.2, "sigma_dx": 0.5, "sigma_dz": 0.5},
    "rotate_y": {"p": 1.0, "sigma_theta": np.deg2rad(60.0)},
}

motion_GRU, info = train_clip_with_split(
    motion_model=motion_GRU,
    motion_ids=motion_ids,
    motion_paths=motion_paths,
    map_df=map_df,
    text_df=text_df,
    text_emb=text_emb,
    text_model_name=text_encoder,
    save_path="mmotion_GRU_512_1024_aug.pth",
    epochs=100,
    batch_size=128,
    time_padding=True,
    augs=augs,
    patience=20,
    n_val_batches=100,
    weight_decay=1e-4,
    grad_clip=1.0,
    scheduler_cfg={"name": "cosine", "T_max": 150, "eta_min": 1e-6},
    lr=5e-4,           # was 1e-3
    warmup_epochs=10,
)

epoch 001/100 | lr=5.01e-05 | train_loss=4.8603 | val_score=0.0738 | MRR=0.1280 | R@1=0.050 R@2=0.070 R@3=0.070 R@5=0.120 R@10=0.250
Modèle sauvegardé (Nouveau meilleur score : 0.0738)
Found New Best Model! (MRR=0.1280)
epoch 002/100 | lr=1.00e-04 | train_loss=4.7863 | val_score=0.2928 | MRR=0.3394 | R@1=0.200 R@2=0.260 R@3=0.350 R@5=0.520 R@10=0.740
Modèle sauvegardé (Nouveau meilleur score : 0.2928)
Found New Best Model! (MRR=0.3394)
epoch 003/100 | lr=1.50e-04 | train_loss=4.6039 | val_score=0.3638 | MRR=0.4012 | R@1=0.230 R@2=0.360 R@3=0.480 R@5=0.640 R@10=0.780
Modèle sauvegardé (Nouveau meilleur score : 0.3638)
Found New Best Model! (MRR=0.4012)
epoch 004/100 | lr=2.00e-04 | train_loss=4.3484 | val_score=0.4161 | MRR=0.4502 | R@1=0.250 R@2=0.450 R@3=0.560 R@5=0.700 R@10=0.860
Modèle sauvegardé (Nouveau meilleur score : 0.4161)
Found New Best Model! (MRR=0.4502)
epoch 005/100 | lr=2.50e-04 | train_loss=4.1539 | val_score=0.4091 | MRR=0.4534 | R@1=0.280 R@2=0.390 R@3=0.530 R@5=0.66

## MDM

In [10]:
augs = {
    "translate_xz": {"p": 0.2, "sigma_dx": 0.5, "sigma_dz": 0.5},
    "rotate_y": {"p": 1.0, "sigma_theta": np.deg2rad(60.0)},
}

mdm_encoder = MDMMotionEncoder(input_dim=d_in, latent_dim=512, nhead=8, num_layers=6)

mdm_encoder, info = train_clip_with_split(
    motion_model=mdm_encoder,
    motion_ids=motion_ids,
    motion_paths=motion_paths,
    map_df=map_df,
    text_df=text_df,
    text_emb=text_emb,
    text_model_name=text_encoder,
    save_path="mdm_encoder_512_v2.pth",
    epochs=100,
    batch_size=32,
    time_padding=True,
    augs=augs,
    patience=30,
    n_val_batches=100,
    lr=5e-5,           # was 1e-4 — stays below NaN threshold, no strike expected
    warmup_epochs=10,
    grad_clip=1.0,
    weight_decay=5e-4, # was 1e-4 — counteracts overfitting seen after epoch 42
    scheduler_cfg={"name": "cosine", "T_max": 90, "eta_min": 1e-6},
)

epoch 001/100 | lr=5.09e-06 | train_loss=3.4788 | val_score=0.0475 | MRR=0.1002 | R@1=0.020 R@2=0.040 R@3=0.070 R@5=0.090 R@10=0.200
Modèle sauvegardé (Nouveau meilleur score : 0.0475)
Found New Best Model! (MRR=0.1002)
epoch 002/100 | lr=1.01e-05 | train_loss=3.4321 | val_score=0.1600 | MRR=0.2129 | R@1=0.070 R@2=0.140 R@3=0.220 R@5=0.350 R@10=0.580
Modèle sauvegardé (Nouveau meilleur score : 0.1600)
Found New Best Model! (MRR=0.2129)
epoch 003/100 | lr=1.51e-05 | train_loss=3.3444 | val_score=0.2137 | MRR=0.2642 | R@1=0.090 R@2=0.220 R@3=0.300 R@5=0.430 R@10=0.700
Modèle sauvegardé (Nouveau meilleur score : 0.2137)
Found New Best Model! (MRR=0.2642)
epoch 004/100 | lr=2.01e-05 | train_loss=3.2705 | val_score=0.2903 | MRR=0.3346 | R@1=0.160 R@2=0.310 R@3=0.370 R@5=0.540 R@10=0.730
Modèle sauvegardé (Nouveau meilleur score : 0.2903)
Found New Best Model! (MRR=0.3346)
epoch 005/100 | lr=2.50e-05 | train_loss=3.1325 | val_score=0.3470 | MRR=0.3922 | R@1=0.240 R@2=0.340 R@3=0.430 R@5=0.54

# Eval & build submission

In [ ]:
pred_df = build_submission_df(trained_clip, test_root="./test/test", top_k=10)
pred_df.head()

In [ ]:
pred_df.to_csv("./submission_bge-m3_LSTM2.csv", index=False)